In [132]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import zipfile
import typing

In [2]:
URL = 'http://armenianhouse.org/tumanyan/tales-ru/nazar.html'

In [3]:
res = requests.get(URL)
res.encoding = res.apparent_encoding
res

<Response [200]>

In [5]:
soup = BeautifulSoup(res.text, 'html.parser')

In [6]:
tags = [a for a in soup.find_all("a") if a.parent.name == "div"]

## Russian data

In [7]:
ru_links = ["http://armenianhouse.org"+ "/tumanyan/tales-ru/" + l["href"] for l in tags]

In [9]:
del ru_links[0]

In [11]:
ru_links.append('http://armenianhouse.org/tumanyan/tales-ru/nazar.html')

In [109]:
ru_links.remove('http://armenianhouse.org/tumanyan/tales-ru/brother.html')

In [110]:
ru_links

['http://armenianhouse.org/tumanyan/tales-ru/fox.html',
 'http://armenianhouse.org/tumanyan/tales-ru/fool.html',
 'http://armenianhouse.org/tumanyan/tales-ru/fish.html',
 'http://armenianhouse.org/tumanyan/tales-ru/kid.html',
 'http://armenianhouse.org/tumanyan/tales-ru/jug.html',
 'http://armenianhouse.org/tumanyan/tales-ru/liar.html',
 'http://armenianhouse.org/tumanyan/tales-ru/rooster.html',
 'http://armenianhouse.org/tumanyan/tales-ru/feast.html',
 'http://armenianhouse.org/tumanyan/tales-ru/master.html',
 'http://armenianhouse.org/tumanyan/tales-ru/tsar.html',
 'http://armenianhouse.org/tumanyan/tales-ru/nazar.html',
 'http://armenianhouse.org/tumanyan/tales-am/nazar.html']

In [103]:
def get_tales(url: str):
  result = {}
  print(url)
  res = requests.get(url)
  res.encoding = res.apparent_encoding
  soup = BeautifulSoup(res.text, 'html.parser')
  try:
    title = soup.h1.text
  except AttributeError:
    title_el = soup.find('p')
    if next(title_el.children).name == 'span':
      title = title_el.text
    else:
      title = ''
  text = BeautifulSoup(" ".join([p.text for p in soup.find_all("p") if p.parent.name != "td" and next(p.children).name is None]), "html.parser").get_text().replace("\xa0", " ")
  result[title]=[text]
  return result

In [111]:
result = [get_tales(l) for l in ru_links]

http://armenianhouse.org/tumanyan/tales-ru/fox.html
http://armenianhouse.org/tumanyan/tales-ru/fool.html
http://armenianhouse.org/tumanyan/tales-ru/fish.html
http://armenianhouse.org/tumanyan/tales-ru/kid.html
http://armenianhouse.org/tumanyan/tales-ru/jug.html
http://armenianhouse.org/tumanyan/tales-ru/liar.html
http://armenianhouse.org/tumanyan/tales-ru/rooster.html
http://armenianhouse.org/tumanyan/tales-ru/feast.html
http://armenianhouse.org/tumanyan/tales-ru/master.html
http://armenianhouse.org/tumanyan/tales-ru/tsar.html
http://armenianhouse.org/tumanyan/tales-ru/nazar.html
http://armenianhouse.org/tumanyan/tales-am/nazar.html


In [112]:
tales = {}
for i in result:
  tales.update(i)

In [113]:
df = pd.DataFrame(ru_links, columns=['url'])

In [114]:
df2 = pd.DataFrame(tales.items(), columns=['title_ru', 'text_ru'])

In [115]:
frames = [df, df2]

In [116]:
ru_tales = pd.concat(frames, axis=1)

## Armenian data

In [119]:
am_links = ['http://armenianhouse.org/tumanyan/tales-am/barekendan.html',
 'http://armenianhouse.org/tumanyan/tales-am/fox.html',
 'http://armenianhouse.org/tumanyan/tales-am/fool.html',
 'http://armenianhouse.org/tumanyan/tales-am/fish.html',
 'http://armenianhouse.org/tumanyan/tales-am/kid.html',
 'http://armenianhouse.org/tumanyan/tales-am/gold.html', #+
 'http://armenianhouse.org/tumanyan/tales-am/liar.html',
 'http://armenianhouse.org/tumanyan/tales-am/rooster.html',
 'http://armenianhouse.org/tumanyan/tales-am/feast.html',
 'http://armenianhouse.org/tumanyan/tales-am/owner.html',#+
 'http://armenianhouse.org/tumanyan/tales-am/king.html', #+
 'http://armenianhouse.org/tumanyan/tales-am/nazar.html']

In [ ]:
result2 = [get_tales(l) for l in am_links]

In [121]:
tales2 = {}
for i in result2:
  tales2.update(i)

In [122]:
df3 = pd.DataFrame(tales2.items(), columns=['title_am', 'text_am'])

In [124]:
frames2 = [ru_tales, df3]

## Merging and downloading

In [125]:
all_tales = pd.concat(frames2, axis=1)

In [128]:
all_tales.to_csv('ru_ar_tales.csv')

In [130]:
zip_filename = "scraped.zip"
zip = zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED)
zip.write("ru_ar_tales.csv")
zip.close()

In [131]:
from google.colab import files

files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>